In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 200)
%matplotlib inline

In [2]:
train = pd.read_csv('train_pre.csv')

In [3]:
train_id = train[['user_id']]
train_y = train[['current_service']]
train_x = train.drop(['user_id', 'current_service'], axis=1)
train_feature = train_x.columns

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_y_le = le.fit_transform(train_y['current_service'])

In [5]:
test = pd.read_csv('test_pre.csv')

In [6]:
test_id = test[['user_id']]
test_x = test.drop(['user_id'], axis=1)

In [7]:
x_train = train_x.values
x_test = test_x.values

In [8]:
from sklearn.model_selection import KFold

ntrain = train_x.shape[0]
ntest = test_x.shape[0]
seed = 0
nfold = 5
kf = KFold(n_splits=nfold, random_state=seed, shuffle=False)

def get_out_fold(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain, ))
    oof_test = np.zeros((ntest, ))
    oof_test_skf = np.empty((nfold, ntest))
    
    for i, (train_index, valid_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]
        x_va = x_train[valid_index]
        y_tr = y_train[train_index]
        
        clf.fit(x_tr, y_tr)
        
        oof_train[valid_index] = clf.predict(x_va)
        oof_test_skf[i, :] = clf.predict(x_test)
        
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [10]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier

rf = RandomForestClassifier(n_estimators=600, max_depth=8, random_state=42, verbose=1, oob_score=True)
gbdt = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05)
# et = ExtraTreesClassifier(n_estimators=200, max_depth=7, random_state=42, verbose=1)
# rf_oof_train, rf_oof_test = get_out_fold(orv_rf, x_train, train_y_le, x_test)
# gb_oof_train, gb_oof_test = get_out_fold(orv_gbdt, x_train, train_y_le, x_test)
# et_oof_train, et_oof_test = get_out_fold(orv_et, x_train, train_y_le, x_test)


In [11]:
score_rf = []
for index, (train_index, valid_index) in enumerate(kf.split(train_x, train_y_le)):
    print(index)
    x_train, x_valid, y_train, y_valid = train_x.loc[train_index], train_x.loc[valid_index], train_y_le[train_index], train_y_le[valid_index]
    clf = gbdt.fit(x_train.values, y_train)
    x_pred = clf.predict(x_valid)
    valid_pre = train_id.loc[valid_index]
    valid_pre['predict_service'] = le.inverse_transform(x_pred)
    validation = train_id.loc[valid_index]
    validation['current_service'] = le.inverse_transform(y_valid)
    score_rf.append(score(validation, valid_pre))
    y_test = clf.predict(test_x)

#     lgb_oof_train[valid_index] = x_pred
    
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

0


NameError: name 'score' is not defined

In [19]:
submit_xgb = np.load('submit_xgb.npy')

In [20]:
submit_xgb = le.inverse_transform(submit_xgb)
df_test = test_id[['user_id']]
df_test['predict_service'] = submit_xgb

In [22]:
df_test.to_csv('xgb_base_0.744.csv', index=None)

In [11]:
def score(tr, te):
    labels = tr['current_service'].unique()
    f1_score = 0
    for label in labels:
        tp = 0
        fp = 0
        fn = 0
        sub_tr = tr[tr['current_service'] == label]['current_service']
        index_tr = tr[tr['current_service'] == label].index.tolist()
        sub_te = te['predict_service'][index_tr]
        for la in sub_te.values:
            if la == label:
                tp = tp + 1
            else:
                fn = fn + 1
        index_te = te[te['predict_service'] == label].index.tolist()
        fp_list = set(index_te).difference(index_tr)
        fp = len(fp_list)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = f1_score + (2*precision*recall)/(precision+recall)
#         print((2*precision*recall)/(precision+recall), label)
    final_score = (1/(len(labels))*f1_score)**2
    
    return final_score